In [ ]:
import tkinter as tk
from tkinter import ttk
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import ib_insync
from ib_insync import Stock
import threading
from datetime import datetime, timedelta
import nest_asyncio
import pandas as pd
from matplotlib.widgets import Button

nest_asyncio.apply()

from ib_insync import IB

ib = IB()
ib.connect('127.0.0.1', 7497, clientId=7)
print("Connesso" if ib.isConnected() else "Non connesso")

class TradingApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Trading Interface")
        self.ib = ib

        # Parametri delle Bande di Bollinger
        self.bollinger_std_dev = 2
        self.bollinger_days = 20
        self.bollinger_active = False  # Variabile per tenere traccia dello stato delle Bande di Bollinger
        self.bollinger_lines = []  # Variabile per tenere traccia delle linee delle Bande di Bollinger

        # Creazione del frame per il grafico
        self.frame_chart = ttk.Frame(root)
        self.frame_chart.grid(row=0, column=0, padx=10, pady=10)

        # Creazione del grafico
        self.fig, self.ax = plt.subplots()
        self.canvas = FigureCanvasTkAgg(self.fig, master=self.frame_chart)
        self.canvas.get_tk_widget().pack()

        # Creazione del frame per i pulsanti delle strategie
        self.frame_buttons = ttk.Frame(root)
        self.frame_buttons.grid(row=0, column=1, padx=10, pady=10)

        # Lista delle strategie di trading
        self.strategies = ["Strategia RSI", "Bande di Bollinger", "Strategia 3", "Strategia 4", "Strategia 5",
                           "Strategia 6", "Strategia 7", "Strategia 8", "Strategia 9", "Strategia 10"]

        # Creazione dei pulsanti per le strategie
        for strategy in self.strategies:
            button = ttk.Button(self.frame_buttons, text=strategy, command=lambda s=strategy: self.activate_strategy(s))
            button.pack(pady=5)

        # Aggiungi pulsante per configurare le Bande di Bollinger
        self.bollinger_config_button = ttk.Button(self.frame_buttons, text="Configura Bande di Bollinger", command=self.configure_bollinger)
        self.bollinger_config_button.pack(pady=5)

        #Aggiungi pulsante per entrare nella schermata dei 10 titoli più importanti di S&P
        self.ten_sp_button = ttk.Button(self.frame_buttons, text="Mostrami i 10 titoli", command=self.show_sp500)
        self.ten_sp_button.pack(pady=10)

        # Scarica i dati della sessione di ieri
        self.yesterday_data = self.get_yesterday_data()

        # Variabile per tenere traccia del tempo
        self.start_time = datetime.now()

        # Thread per aggiornare il grafico in tempo reale
        self.thread = threading.Thread(target=self.update_chart)
        self.thread.start()

    def activate_strategy(self, strategy):
        print(f"Attivata {strategy}")
        if strategy == "Strategia RSI":
            self.plot_rsi()
        elif strategy == "Bande di Bollinger":
            self.plot_bollinger()

    def get_yesterday_data(self):
        contract = Stock('AAPL', 'SMART', 'USD')
        end_time = datetime.now().replace(hour=22, minute=0, second=0, microsecond=0) - timedelta(days=1)
        end_time_str = end_time.strftime('%Y%m%d %H:%M:%S')
        print(f"End time: {end_time_str}")
        bars = self.ib.reqHistoricalData(contract, endDateTime=end_time_str, durationStr='1 D', barSizeSetting='1 min', whatToShow='MIDPOINT', useRTH=True, formatDate=2, timeout=120)
        if bars:
            print(f"Number of bars received: {len(bars)}")
            return [bar.close for bar in bars]
        else:
            print("No bars received")
            return []

    def update_chart(self):
        contract = Stock('AAPL', 'SMART', 'USD')
        while True:
            try:
                # Aggiorna il grafico con i dati di ieri
                self.root.after(0, self.plot_data, self.yesterday_data)

                # Ottieni i dati in tempo reale
                self.ib.reqMktData(contract, '', False, False)
                self.ib.sleep(60)  # Attendi un minuto

                # Aggiorna il grafico con i nuovi dati
                self.root.after(0, self.plot_data, self.yesterday_data + [self.ib.ticker(contract).last])
            except Exception as e:
                print(f"Errore: {e}")


    def plot_bollinger(self):
        if self.bollinger_active:
            # Cancella le bande dal grafico
            for line in self.bollinger_lines:
                line.remove()
            self.bollinger_lines = []
            self.canvas.draw()
            self.bollinger_active = False
        else:
            # Ottieni i dati di chiusura
            data = pd.Series(self.yesterday_data)
            upper_band, middle_band, lower_band = self.calculate_bollinger_bands(data)
            self.bollinger_lines = [
                self.ax.plot(upper_band, label='Upper Band')[0],
                self.ax.plot(middle_band, label='Middle Band')[0],
                self.ax.plot(lower_band, label='Lower Band')[0]
            ]
            self.canvas.draw()
            self.bollinger_active = True

    def calculate_bollinger_bands(self, data, window=20, num_of_std=2):
        rolling_mean = data.rolling(window=window).mean()
        rolling_std = data.rolling(window=window).std()
        upper_band = rolling_mean + (rolling_std * num_of_std)
        lower_band = rolling_mean - (rolling_std * num_of_std)
        return upper_band, rolling_mean, lower_band

    def configure_bollinger(self):
        config_window = tk.Toplevel(self.root)
        config_window.title("Configura Bande di Bollinger")

        tk.Label(config_window, text="Numero di giorni:").pack(pady=5)
        days_entry = tk.Entry(config_window)
        days_entry.pack(pady=5)
        days_entry.insert(0, str(self.bollinger_days))

        tk.Label(config_window, text="Deviazione standard:").pack(pady=5)
        std_dev_entry = tk.Entry(config_window)
        std_dev_entry.pack(pady=5)
        std_dev_entry.insert(0, str(self.bollinger_std_dev))

        def save_config():
            self.bollinger_days = int(days_entry.get())
            self.bollinger_std_dev = float(std_dev_entry.get())
            config_window.destroy()

        save_button = ttk.Button(config_window, text="Salva", command=save_config)
        save_button.pack(pady=5)


    def plot_data(self, data):
        self.ax.clear()
        self.ax.plot(data[:len(self.yesterday_data)], label='Apple')
        
        # Calcola il tempo trascorso dall'inizio della sessione di oggi
        elapsed_time = (datetime.now() - self.start_time).total_seconds() / 60
        
        # Se il tempo trascorso è inferiore a 10 minuti, rendi i dati in tempo reale tratteggiati e verdi
        if elapsed_time <= 10:
            self.ax.plot(range(len(self.yesterday_data), len(data)), data[len(self.yesterday_data):], 'g--', label='Realtime')
        else:
            self.ax.plot(range(len(self.yesterday_data), len(data)), data[len(self.yesterday_data):], 'g-', label='Realtime')
        
        self.ax.legend()
        self.canvas.draw()

    def plot_rsi(self):
        # Calcola l'RSI
        rsi_length = 21
        buy_line = 65
        exit_line = 50
        close_prices = self.yesterday_data
        rsi = self.calculate_rsi(close_prices, rsi_length)
    

        # Sovrapponi il grafico dell'RSI
        self.ax.plot(rsi, label='RSI', color='blue')
        self.ax.axhline(y=buy_line, color='blue', linestyle='--', label='Buy Line')
        self.ax.axhline(y=exit_line, color='purple', linestyle='--', label='Exit Line')
        
        self.ax.legend()
        self.canvas.draw()

    def calculate_rsi(data, window=14):
        delta = data.diff()
        gain = (delta.where(delta > 0, 0)).fillna(0)
        loss = (-delta.where(delta < 0, 0)).fillna(0)
    
        avg_gain = gain.rolling(window=window, min_periods=1).mean()
        avg_loss = loss.rolling(window=window, min_periods=1).mean()
    
        rs = avg_gain / avg_loss
        rsi = 100 - (100 / (1 + rs))
        return rsi

    def show_sp500(self):
        new_window = tk.Toplevel(self.root)
        new_window.title("S&P 500")
    
        screen_width = self.root.winfo_screenwidth()
        screen_height = self.root.winfo_screenheight()
        new_window.geometry(f"{screen_width}x{screen_height}")
    
        canvas = tk.Canvas(new_window)
        scrollbar = tk.Scrollbar(new_window, orient="vertical", command=canvas.yview)
        scrollable_frame = ttk.Frame(canvas)
    
        scrollable_frame.bind(
            "<Configure>",
            lambda e: canvas.configure(
                scrollregion=canvas.bbox("all")
            )
        )
    
        canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")
        canvas.configure(yscrollcommand=scrollbar.set)
    
        canvas.pack(side="left", fill="both", expand=True)
        scrollbar.pack(side="right", fill="y")
    
        for i, symbol in enumerate(["AAPL", "MSFT", "GOOGL", "AMZN", "META", "TSLA", "BRK B", "JNJ", "V", "WMT"]):
            frame = ttk.Frame(scrollable_frame)
            frame.grid(row=i//2, column=i%2, padx=10, pady=10)
    
            fig, ax = plt.subplots()
            canvas = FigureCanvasTkAgg(fig, master=frame)
            canvas.get_tk_widget().pack()
    
            # Ottieni i dati in tempo reale e plottali
            data = self.get_realtime_data(symbol)
            if not data.empty:
                ax.plot(data['date'], data['close'], label=symbol)
                ax.legend()
                canvas.draw()
    
            # Aggiungi i pulsanti di zoom
            self.add_zoom_buttons(ax)

    def get_realtime_data(self, symbol):
        contract = Stock(symbol, 'SMART', 'USD')
        end_time = datetime.now()
        start_time = end_time - timedelta(days=1)
        bars = self.ib.reqHistoricalData(
            contract,
            endDateTime=end_time,
            durationStr='1 D',
            barSizeSetting='1 min',
            whatToShow='MIDPOINT',
            useRTH=True,
            formatDate=1
        )
        if bars:
            data = pd.DataFrame(bars)
            return data
        else:
            print("No data received")
            return pd.DataFrame()
    def zoom_factory(ax, base_scale=2.):
        def zoom_fun(event):
            cur_xlim = ax.get_xlim()
            cur_ylim = ax.get_ylim()
            xdata = event.xdata
            ydata = event.ydata
            if event.button == 'up':
                scale_factor = 1 / base_scale
            elif event.button == 'down':
                scale_factor = base_scale
            else:
                scale_factor = 1
                print(event.button)
            new_width = (cur_xlim[1] - cur_xlim[0]) * scale_factor
            new_height = (cur_ylim[1] - cur_ylim[0]) * scale_factor
            relx = (cur_xlim[1] - xdata) / (cur_xlim[1] - cur_xlim[0])
            rely = (cur_ylim[1] - ydata) / (cur_ylim[1] - cur_ylim[0])
            ax.set_xlim([xdata - new_width * (1 - relx), xdata + new_width * (relx)])
            ax.set_ylim([ydata - new_height * (1 - rely)])
            ax.figure.canvas.draw()
    
        fig = ax.get_figure()
        fig.canvas.mpl_connect('scroll_event', zoom_fun)
        return zoom_fun

    def add_zoom_buttons(self,ax):
        fig = ax.get_figure()
        zoom_in_ax = fig.add_axes([0.8, 0.05, 0.1, 0.075])
        zoom_out_ax = fig.add_axes([0.8, 0.15, 0.1, 0.075])
        zoom_in_button = Button(zoom_in_ax, 'Zoom In')
        zoom_out_button = Button(zoom_out_ax, 'Zoom Out')
    
        def zoom_in(event):
            zoom_factory(ax, base_scale=1.5)(event)
    
        def zoom_out(event):
            zoom_factory(ax, base_scale=0.5)(event)
    
        zoom_in_button.on_clicked(zoom_in)
        zoom_out_button.on_clicked(zoom_out)


if __name__ == "__main__":
    root = tk.Tk()
    app = TradingApp(root)
    root.mainloop()


Connesso
End time: 20240818 22:00:00
Number of bars received: 390


Error 200, reqId 11: Nessuna descrizione di titoli trovata per la richiesta, contract: Stock(symbol='BRK.B', exchange='SMART', currency='USD')


No data received


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\admin\anaconda3\Lib\tkinter\__init__.py", line 1967, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\admin\AppData\Local\Temp\ipykernel_14564\2610761311.py", line 219, in <lambda>
    lambda e: canvas.configure(
              ^^^^^^^^^^^^^^^^
AttributeError: 'FigureCanvasTkAgg' object has no attribute 'configure'. Did you mean: 'figure'?
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\admin\anaconda3\Lib\tkinter\__init__.py", line 1967, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\admin\AppData\Local\Temp\ipykernel_14564\2610761311.py", line 219, in <lambda>
    lambda e: canvas.configure(
              ^^^^^^^^^^^^^^^^
AttributeError: 'FigureCanvasTkAgg' object has no attribute 'configure'. Did you mean: 'figure'?
Error 200, reqId 34: Nessuna descrizione di titoli trovata per la richiesta, contr

No data received


C:\Users\admin\AppData\Local\Temp\ipykernel_14564\2610761311.py:234: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, ax = plt.subplots()
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\admin\anaconda3\Lib\tkinter\__init__.py", line 1967, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\admin\AppData\Local\Temp\ipykernel_14564\2610761311.py", line 219, in <lambda>
    lambda e: canvas.configure(
              ^^^^^^^^^^^^^^^^
AttributeError: 'FigureCanvasTkAgg' object has no attribute 'configure'. Did you mean: 'figure'?
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\admin\anaconda3\Lib\tkinter\__init__.py", line 196